<a href="https://colab.research.google.com/github/sandeepgundeboina/Preppin-Data/blob/main/DeltaLake.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install delta-spark==2.0.0

import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .appName("DeltaTableExample") \
    .config("spark.jars.packages", "io.delta:delta-core_2.12:2.0.0") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .getOrCreate()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.5/281.5 MB 1.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 kB 12.0 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.2.4-py2.py3-none-any.whl size=282040916 sha256=85ca7e40be5bb6534b61ea7fa7a991f56f9b45abe6cfd8feb5fb60fdf1024f21
  Stored in directory: /root/.cache/pip/wheels/e7/e3/c8/c358dac750f2b6a4b03328d10e05a5c69501664bd6504b6c3e
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7
  Attempting uninstall: pyspark
    Found existing installation: pyspark 3.5.3
    Uninstalling pyspark-3.5.3:
      Successfully uninstalled pyspark-3.5.3


In [ ]:
from delta.tables import *
DeltaTable.createOrReplace(spark) \
          .tableName("delta_table") \
          .addColumn("id", "INT") \
          .addColumn("name", "STRING") \
          .addColumn("Subject", "String") \
          .addColumn('MarksObtained', 'Int') \
          .addColumn('Percentage', 'Float')\
          .property('description','For Demo Purpose')\
          .location('/content/sample_data/delta_table')\
          .execute()

In [ ]:

!cat /content/sample_data/delta_table/_delta_log/00000000000000000000.json | head -5

{"protocol":{"minReaderVersion":1,"minWriterVersion":2}}
{"metaData":{"id":"bf4041bc-8958-4bbb-9831-6be594749d51","format":{"provider":"parquet","options":{}},"schemaString":"{\"type\":\"struct\",\"fields\":[{\"name\":\"id\",\"type\":\"integer\",\"nullable\":true,\"metadata\":{}},{\"name\":\"name\",\"type\":\"string\",\"nullable\":true,\"metadata\":{}},{\"name\":\"Subject\",\"type\":\"string\",\"nullable\":true,\"metadata\":{}},{\"name\":\"MarksObtained\",\"type\":\"integer\",\"nullable\":true,\"metadata\":{}},{\"name\":\"Percentage\",\"type\":\"float\",\"nullable\":true,\"metadata\":{}}]}","partitionColumns":[],"configuration":{"description":"For Demo Purpose"},"createdTime":1731317902441}}
{"commitInfo":{"timestamp":1731317903384,"operation":"CREATE OR REPLACE TABLE","operationParameters":{"isManaged":"false","description":null,"partitionBy":"[]","properties":"{\"description\":\"For Demo Purpose\"}"},"isolationLevel":"Serializable","isBlindAppend":true,"operationMetrics":{},"engineIn

In [ ]:
delta_table = spark.read.format("delta").load("/content/sample_data/delta_table")

In [ ]:
delta_table.show()

+---+----+-------+-------------+----------+
| id|name|Subject|MarksObtained|Percentage|
+---+----+-------+-------------+----------+
+---+----+-------+-------------+----------+



In [ ]:
# Create a new DataFrame with new records, explicitly defining schema
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, FloatType

new_data = [(1,'Sandeep','Maths',907,90.7)]
schema = StructType([
    StructField("id", IntegerType(), True),
    StructField("name", StringType(), True),
    StructField("Subject", StringType(), True),
    StructField("MarksObtained", IntegerType(), True),
    StructField("Percentage", FloatType(), True)
])
new_df = spark.createDataFrame(new_data, schema)

# Append the new DataFrame to the existing Delta table
new_df.write.format("delta").mode("append").save("/content/sample_data/delta_table")
updated_delta_df = spark.read.format("delta").load('/content/sample_data/delta_table')
updated_delta_df.show()

+---+-------+-------+-------------+----------+
| id|   name|Subject|MarksObtained|Percentage|
+---+-------+-------+-------------+----------+
|  1|Sandeep|  Maths|          907|      90.7|
+---+-------+-------+-------------+----------+



In [ ]:
delta_table.show()

+---+-------+-------+-------------+----------+
| id|   name|Subject|MarksObtained|Percentage|
+---+-------+-------+-------------+----------+
|  1|Sandeep|  Maths|          907|      90.7|
+---+-------+-------+-------------+----------+



In [ ]:
!cat /content/sample_data/delta_table/_delta_log/00000000000000000001.json | head -5

{"add":{"path":"part-00000-c7f315d5-e50c-48f7-abf3-de9a5df7f9e6-c000.snappy.parquet","partitionValues":{},"size":652,"modificationTime":1731317932947,"dataChange":true,"stats":"{\"numRecords\":0,\"minValues\":{},\"maxValues\":{},\"nullCount\":{}}"}}
{"add":{"path":"part-00001-2ba15b3f-7f73-4331-862d-d6f0efc3f028-c000.snappy.parquet","partitionValues":{},"size":1448,"modificationTime":1731317932965,"dataChange":true,"stats":"{\"numRecords\":1,\"minValues\":{\"id\":1,\"name\":\"Sandeep\",\"Subject\":\"Maths\",\"MarksObtained\":907,\"Percentage\":90.7},\"maxValues\":{\"id\":1,\"name\":\"Sandeep\",\"Subject\":\"Maths\",\"MarksObtained\":907,\"Percentage\":90.7},\"nullCount\":{\"id\":0,\"name\":0,\"Subject\":0,\"MarksObtained\":0,\"Percentage\":0}}"}}
{"commitInfo":{"timestamp":1731317933235,"operation":"WRITE","operationParameters":{"mode":"Append","partitionBy":"[]"},"readVersion":0,"isolationLevel":"Serializable","isBlindAppend":true,"operationMetrics":{"numFiles":"2","numOutputRows":"1"

In [ ]:
new_data = [(2,'Rahul','Maths',924,92.4),(3,'Spandana',"Maths",963,96.3),(4,'Pradeep','Maths',948,94.8)]
schema = StructType([
    StructField("id", IntegerType(), True),
    StructField("name", StringType(), True),
    StructField("Subject", StringType(), True),
    StructField("MarksObtained", IntegerType(), True),
    StructField("Percentage", FloatType(), True)
])
new_df = spark.createDataFrame(new_data, schema)

# Append the new DataFrame to the existing Delta table
new_df.write.format("delta").mode("append").save("/content/sample_data/delta_table")
updated_delta_df = spark.read.format("delta").load('/content/sample_data/delta_table')
updated_delta_df.show()

+---+--------+-------+-------------+----------+
| id|    name|Subject|MarksObtained|Percentage|
+---+--------+-------+-------------+----------+
|  3|Spandana|  Maths|          963|      96.3|
|  4| Pradeep|  Maths|          948|      94.8|
|  1| Sandeep|  Maths|          907|      90.7|
|  2|   Rahul|  Maths|          924|      92.4|
+---+--------+-------+-------------+----------+



In [ ]:
delta_table

DataFrame[id: int, name: string, Subject: string, MarksObtained: int, Percentage: float]

In [ ]:
!cat /content/sample_data/delta_table/_delta_log/00000000000000000003.json

cat: /content/sample_data/delta_table/_delta_log/00000000000000000003.json: No such file or directory


In [ ]:
from delta.tables import *

# Assuming your Delta table is located at '/content/sample_data/delta_table'
delta_table_instance = DeltaTable.forPath(spark, '/content/sample_data/delta_table')



In [ ]:
delta_table_instance.toDF().show()

+---+--------+-------+-------------+----------+
| id|    name|Subject|MarksObtained|Percentage|
+---+--------+-------+-------------+----------+
|  3|Spandana|  Maths|          963|      96.3|
|  4| Pradeep|  Maths|          948|      94.8|
|  1| Sandeep|  Maths|          907|      90.7|
|  2|   Rahul|  Maths|          924|      92.4|
+---+--------+-------+-------------+----------+



In [ ]:
delta_table_instance.delete("id = 2")

In [ ]:
delta_table_instance.toDF().show()

+---+--------+-------+-------------+----------+
| id|    name|Subject|MarksObtained|Percentage|
+---+--------+-------+-------------+----------+
|  3|Spandana|  Maths|          963|      96.3|
|  4| Pradeep|  Maths|          948|      94.8|
|  1| Sandeep|  Maths|          907|      90.7|
+---+--------+-------+-------------+----------+



In [ ]:
f=spark.read.format('delta').load('/content/sample_data/delta_table')
f.show()

+---+--------+-------+-------------+----------+
| id|    name|Subject|MarksObtained|Percentage|
+---+--------+-------+-------------+----------+
|  3|Spandana|  Maths|          963|      96.3|
|  4| Pradeep|  Maths|          948|      94.8|
|  1| Sandeep|  Maths|          907|      90.7|
+---+--------+-------+-------------+----------+



In [ ]:
delta_instance=DeltaTable.forName(spark,'delta_table')
delta_instance.toDF().show()

+---+--------+-------+-------------+----------+
| id|    name|Subject|MarksObtained|Percentage|
+---+--------+-------+-------------+----------+
|  3|Spandana|  Maths|          963|      96.3|
|  4| Pradeep|  Maths|          948|      94.8|
|  1| Sandeep|  Maths|          907|      90.7|
+---+--------+-------+-------------+----------+



In [ ]:
delta_instance.history().show()

+-------+--------------------+------+--------+--------------------+--------------------+----+--------+---------+-----------+--------------+-------------+--------------------+------------+--------------------+
|version|           timestamp|userId|userName|           operation| operationParameters| job|notebook|clusterId|readVersion|isolationLevel|isBlindAppend|    operationMetrics|userMetadata|          engineInfo|
+-------+--------------------+------+--------+--------------------+--------------------+----+--------+---------+-----------+--------------+-------------+--------------------+------------+--------------------+
|      3|2024-11-11 09:39:...|  null|    null|              DELETE|{predicate -> ["(...|null|    null|     null|          2|  Serializable|        false|{numRemovedFiles ...|        null|Apache-Spark/3.2....|
|      2|2024-11-11 09:39:...|  null|    null|               WRITE|{mode -> Append, ...|null|    null|     null|          1|  Serializable|         true|{numFiles -

In [ ]:
new_data1 = [(6,'Advik','Games',999,99.9),(7,'Aashritha','Cooking',954,95.4)]
schema = StructType([
    StructField("id", IntegerType(), True),
    StructField("name", StringType(), True),
    StructField("Subject", StringType(), True),
    StructField("MarksObtained", IntegerType(), True),
    StructField("Percentage", FloatType(), True)
])
new_df = spark.createDataFrame(new_data1, schema)

In [ ]:
new_df.write.insertInto('delta_table',overwrite=False)

In [ ]:
delta_instance.toDF().show()

+---+---------+-------+-------------+----------+
| id|     name|Subject|MarksObtained|Percentage|
+---+---------+-------+-------------+----------+
|  3| Spandana|  Maths|          963|      96.3|
|  4|  Pradeep|  Maths|          948|      94.8|
|  7|Aashritha|Cooking|          954|      95.4|
|  1|  Sandeep|  Maths|          907|      90.7|
|  6|    Advik|  Games|          999|      99.9|
+---+---------+-------+-------------+----------+



In [ ]:
spark.sql('select * from delta_table').show()

+---+---------+-------+-------------+----------+
| id|     name|Subject|MarksObtained|Percentage|
+---+---------+-------+-------------+----------+
|  3| Spandana|  Maths|          963|      96.3|
|  4|  Pradeep|  Maths|          948|      94.8|
|  7|Aashritha|Cooking|          954|      95.4|
|  1|  Sandeep|  Maths|          907|      90.7|
|  6|    Advik|  Games|          999|      99.9|
+---+---------+-------+-------------+----------+



In [ ]:
new_data1 = [(8,'Ram','Science',987,98.7)]
schema = StructType([
    StructField("id", IntegerType(), True),
    StructField("name", StringType(), True),
    StructField("Subject", StringType(), True),
    StructField("MarksObtained", IntegerType(), True),
    StructField("Percentage", FloatType(), True)
])
new_df = spark.createDataFrame(new_data1, schema)

In [ ]:
new_df.write.format('delta').mode('append').saveAsTable('delta_table')

In [ ]:
spark.sql('select * from delta_table').show()

+---+---------+-------+-------------+----------+
| id|     name|Subject|MarksObtained|Percentage|
+---+---------+-------+-------------+----------+
|  3| Spandana|  Maths|          963|      96.3|
|  4|  Pradeep|  Maths|          948|      94.8|
|  7|Aashritha|Cooking|          954|      95.4|
|  1|  Sandeep|  Maths|          907|      90.7|
|  6|    Advik|  Games|          999|      99.9|
|  8|      Ram|Science|          987|      98.7|
+---+---------+-------+-------------+----------+



In [ ]:
spark.sql("delete from delta_table where id=8")

DataFrame[]

In [ ]:
spark.sql('select * from delta_table').show()

+---+---------+-------+-------------+----------+
| id|     name|Subject|MarksObtained|Percentage|
+---+---------+-------+-------------+----------+
|  3| Spandana|  Maths|          963|      96.3|
|  4|  Pradeep|  Maths|          948|      94.8|
|  7|Aashritha|Cooking|          954|      95.4|
|  1|  Sandeep|  Maths|          907|      90.7|
|  6|    Advik|  Games|          999|      99.9|
+---+---------+-------+-------------+----------+



In [ ]:
spark.sql("update delta_table set Percentage=95 where Subject='Cooking'")

DataFrame[]

In [ ]:
spark.sql('select * from delta_table').show()

+---+---------+-------+-------------+----------+
| id|     name|Subject|MarksObtained|Percentage|
+---+---------+-------+-------------+----------+
|  3| Spandana|  Maths|          963|      96.3|
|  4|  Pradeep|  Maths|          948|      94.8|
|  7|Aashritha|Cooking|          954|      95.0|
|  1|  Sandeep|  Maths|          907|      90.7|
|  6|    Advik|  Games|          999|      99.9|
+---+---------+-------+-------------+----------+



In [ ]:
delta_instance.update(
    condition="name='Sandeep'",
    set={"Percentage": "90.8"}
)

In [ ]:
spark.sql('select * from delta_table').show()

+---+---------+-------+-------------+----------+
| id|     name|Subject|MarksObtained|Percentage|
+---+---------+-------+-------------+----------+
|  3| Spandana|  Maths|          963|      96.3|
|  4|  Pradeep|  Maths|          948|      94.8|
|  7|Aashritha|Cooking|          954|      95.0|
|  1|  Sandeep|  Maths|          907|      90.8|
|  6|    Advik|  Games|          999|      99.9|
+---+---------+-------+-------------+----------+



In [ ]:
delta_table = spark.read.format("delta").load("/content/sample_data/delta_table")

In [ ]:
from delta.tables import *
from pyspark.sql.functions import *

# Load the Delta table using DeltaTable
delta_table = DeltaTable.forPath(spark, "/content/sample_data/delta_table")

# Perform the update
delta_table.update(
    condition=col("name")=='Sandeep',
    set={"Percentage": "90.7"}
)

In [ ]:
spark.sql('select * from delta_table').show()

+---+---------+-------+-------------+----------+
| id|     name|Subject|MarksObtained|Percentage|
+---+---------+-------+-------------+----------+
|  3| Spandana|  Maths|          963|      96.3|
|  4|  Pradeep|  Maths|          948|      94.8|
|  7|Aashritha|Cooking|          954|      95.0|
|  1|  Sandeep|  Maths|          907|      90.7|
|  6|    Advik|  Games|          999|      99.9|
+---+---------+-------+-------------+----------+



    Merge

In [ ]:
from pyspark.sql.types import *
from pyspark.sql.functions import *
schema = StructType(
    [
        StructField("id", IntegerType(), True),
        StructField("name", StringType(), True),
        StructField("Subject", StringType(), True),
        StructField("MarksObtained", IntegerType(), True),
        StructField("Percentage", FloatType(), True)
    ])

In [ ]:
data=[(1,'Aashritha','Fun',100,99.9)]
df=spark.createDataFrame(data,schema)
df.show()

+---+---------+-------+-------------+----------+
| id|     name|Subject|MarksObtained|Percentage|
+---+---------+-------+-------------+----------+
|  1|Aashritha|    Fun|          100|      99.9|
+---+---------+-------+-------------+----------+



In [ ]:
from delta.tables import *
from pyspark.sql.functions import *

In [ ]:
df.write.format('delta').mode('append').save('content/sample_data/delta_table_merge')

In [ ]:
merge_read=spark.read.format('delta').load('content/sample_data/delta_table_merge')


In [ ]:
merge_read.show()

+---+---------+-------+-------------+----------+
| id|     name|Subject|MarksObtained|Percentage|
+---+---------+-------+-------------+----------+
|  1|Aashritha|    Fun|          100|      99.9|
+---+---------+-------+-------------+----------+



In [ ]:
new_data=[(1,'Aashritha','Fun',99,99.0),(2,'Advik','Games',97,97.0)]
schema = StructType(
    [
        StructField("id", IntegerType(), True),
        StructField("name", StringType(), True),
        StructField("Subject", StringType(), True),
        StructField("MarksObtained", IntegerType(), True),
        StructField("Percentage", FloatType(), True)
    ])
new_df=spark.createDataFrame(new_data,schema)

In [ ]:
delta_df=DeltaTable.forPath(spark,'content/sample_data/delta_table_merge')

In [ ]:
delta_df.alias('target').merge(
    new_df.alias('source'),
    'target.id = source.id'
).whenMatchedUpdate(set=
                    {
                        'id': 'source.id',
                        'name': 'source.name',
                        'Subject': 'source.Subject',
                        'MarksObtained': 'source.MarksObtained',
                        'Percentage': 'source.Percentage'
                    }).whenNotMatchedInsert(values=
                                            {
                                                'id': 'source.id',
                                                'name': 'source.name',
                                                'Subject': 'source.Subject',
                                                'MarksObtained': 'source.MarksObtained',
                                                'Percentage': 'source.Percentage'
                                            }).execute()

In [ ]:
merge_read=spark.read.format('delta').load('content/sample_data/delta_table_merge')

In [ ]:
merge_read.show()

+---+---------+-------+-------------+----------+
| id|     name|Subject|MarksObtained|Percentage|
+---+---------+-------+-------------+----------+
|  1|Aashritha|    Fun|           99|      99.0|
|  2|    Advik|  Games|           97|      97.0|
+---+---------+-------+-------------+----------+



In [ ]:
lastOperation=delta_df.history()

In [ ]:
explode_df=lastOperation.select(lastOperation.operation,lastOperation.timestamp,explode(lastOperation.operationMetrics))
explode_df_select=explode_df.select('operation','timestamp',explode_df.key,explode_df.value.cast('int'))
explode_df_select.show()

+---------+--------------------+--------------------+-----+
|operation|           timestamp|                 key|value|
+---------+--------------------+--------------------+-----+
|    MERGE|2024-11-11 09:40:...| numTargetRowsCopied|    0|
|    MERGE|2024-11-11 09:40:...|numTargetRowsDeleted|    0|
|    MERGE|2024-11-11 09:40:...| numTargetFilesAdded|    1|
|    MERGE|2024-11-11 09:40:...|     executionTimeMs| 3033|
|    MERGE|2024-11-11 09:40:...|numTargetRowsInse...|    1|
|    MERGE|2024-11-11 09:40:...|          scanTimeMs| 1908|
|    MERGE|2024-11-11 09:40:...|numTargetRowsUpdated|    1|
|    MERGE|2024-11-11 09:40:...|       numOutputRows|    2|
|    MERGE|2024-11-11 09:40:...|numTargetChangeFi...|    0|
|    MERGE|2024-11-11 09:40:...|       numSourceRows|    2|
|    MERGE|2024-11-11 09:40:...|numTargetFilesRem...|    1|
|    MERGE|2024-11-11 09:40:...|       rewriteTimeMs| 1119|
|    WRITE|2024-11-11 09:40:...|            numFiles|    2|
|    WRITE|2024-11-11 09:40:...|       n

In [ ]:
pivot_df=explode_df_select.groupBy('operation').pivot('key').sum('value')
pivot_df.show()

+---------+---------------+--------+--------------+-------------+-------------+-------------------------+-------------------+---------------------+-------------------+--------------------+---------------------+--------------------+-------------+----------+
|operation|executionTimeMs|numFiles|numOutputBytes|numOutputRows|numSourceRows|numTargetChangeFilesAdded|numTargetFilesAdded|numTargetFilesRemoved|numTargetRowsCopied|numTargetRowsDeleted|numTargetRowsInserted|numTargetRowsUpdated|rewriteTimeMs|scanTimeMs|
+---------+---------------+--------+--------------+-------------+-------------+-------------------------+-------------------+---------------------+-------------------+--------------------+---------------------+--------------------+-------------+----------+
|    MERGE|           3033|    null|          null|            2|            2|                        0|                  1|                    1|                  0|                   0|                    1|                   

In [ ]:
merge_table1=DeltaTable.forPath(spark,'content/sample_data/delta_table_merge')
merge_table1.history()

DataFrame[version: bigint, timestamp: timestamp, userId: string, userName: string, operation: string, operationParameters: map<string,string>, job: struct<jobId:string,jobName:string,runId:string,jobOwnerId:string,triggerType:string>, notebook: struct<notebookId:string>, clusterId: string, readVersion: bigint, isolationLevel: string, isBlindAppend: boolean, operationMetrics: map<string,string>, userMetadata: string, engineInfo: string]

In [ ]:
df=spark.read.format('delta').option('versionAsOf',2).load('/content/sample_data/delta_table')

In [ ]:
df.show()

+---+--------+-------+-------------+----------+
| id|    name|Subject|MarksObtained|Percentage|
+---+--------+-------+-------------+----------+
|  3|Spandana|  Maths|          963|      96.3|
|  4| Pradeep|  Maths|          948|      94.8|
|  1| Sandeep|  Maths|          907|      90.7|
|  2|   Rahul|  Maths|          924|      92.4|
+---+--------+-------+-------------+----------+



In [ ]:
spark.sql('select * from delta_table ').show()

+---+---------+-------+-------------+----------+
| id|     name|Subject|MarksObtained|Percentage|
+---+---------+-------+-------------+----------+
|  3| Spandana|  Maths|          963|      96.3|
|  4|  Pradeep|  Maths|          948|      94.8|
|  7|Aashritha|Cooking|          954|      95.0|
|  1|  Sandeep|  Maths|          907|      90.7|
|  6|    Advik|  Games|          999|      99.9|
+---+---------+-------+-------------+----------+



In [ ]:
data=[(1,1),(2,2),(3,3),(4,3)]
schema = StructType(
    [
        StructField("id", IntegerType(), True),
        StructField("customerID", IntegerType(), True),

    ])

df=spark.createDataFrame(
    data,schema)
df.show()

+---+----------+
| id|customerID|
+---+----------+
|  1|         1|
|  2|         2|
|  3|         3|
|  4|         3|
+---+----------+



In [ ]:
df.write.format('delta').mode('overwrite').save('content/sample_data/order')

In [ ]:
employee = spark.read.format("delta").load("/content/content/sample_data/order")

In [ ]:
# Register the DataFrame as a temporary view
employee.createOrReplaceTempView("employee")

# Now you can query the data using SQL
spark.sql('select * from employee').show()

+---+----------+
| id|customerID|
+---+----------+
|  3|         3|
|  4|         3|
|  1|         1|
|  2|         2|
+---+----------+



In [ ]:
spark.sql('select customerID,count(1) noOfOrder from employee group by customerID order by noOfOrder desc limit 1').show()

+----------+---------+
|customerID|noOfOrder|
+----------+---------+
|         3|        2|
+----------+---------+



In [ ]:
df=DeltaTable.forPath(spark,'/content/sample_data/delta_table')

In [ ]:
df.history().show()

+-------+--------------------+------+--------+--------------------+--------------------+----+--------+---------+-----------+--------------+-------------+--------------------+------------+--------------------+
|version|           timestamp|userId|userName|           operation| operationParameters| job|notebook|clusterId|readVersion|isolationLevel|isBlindAppend|    operationMetrics|userMetadata|          engineInfo|
+-------+--------------------+------+--------+--------------------+--------------------+----+--------+---------+-----------+--------------+-------------+--------------------+------------+--------------------+
|     10|2024-11-11 09:50:...|  null|    null|             RESTORE|{version -> 8, ti...|null|    null|     null|          9|  Serializable|        false|{numRestoredFiles...|        null|Apache-Spark/3.2....|
|      9|2024-11-11 09:40:...|  null|    null|              UPDATE|{predicate -> (na...|null|    null|     null|          8|  Serializable|        false|{numRemoved

In [ ]:
df.restoreToVersion(8)

DataFrame[table_size_after_restore: bigint, num_of_files_after_restore: bigint, num_removed_files: bigint, num_restored_files: bigint, removed_files_size: bigint, restored_files_size: bigint]

In [ ]:
spark.sql('select * from delta_table').show()

+---+---------+-------+-------------+----------+
| id|     name|Subject|MarksObtained|Percentage|
+---+---------+-------+-------------+----------+
|  3| Spandana|  Maths|          963|      96.3|
|  4|  Pradeep|  Maths|          948|      94.8|
|  7|Aashritha|Cooking|          954|      95.0|
|  1|  Sandeep|  Maths|          907|      90.8|
|  6|    Advik|  Games|          999|      99.9|
+---+---------+-------+-------------+----------+

